### Engineers' Salary Prediction Challenge

In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats import mannwhitneyu
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

#### Wczytanie danych

In [ ]:
train = pd.read_csv(os.path.join('data', 'train.csv'))
test = pd.read_csv(os.path.join('data', 'test.csv'))

train.head()

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
def compare_train_test(column):
    train_values = train[column].value_counts().reset_index(name='train_count')
    test_values = test[column].value_counts().reset_index(name='test_count')
    return pd.merge(test_values, train_values, how='outer', on=column)

c_job_title = compare_train_test('job_title')
c_job_state = compare_train_test('job_state')

### <center>Braki w kolumnach</center>

In [ ]:
train.isnull().sum().sort_values(ascending=False)

In [ ]:
test.isnull().sum().sort_values(ascending=False)

### <center>Analiza kolumn job_desc</center>

In [ ]:
job_desc_columns = [col for col in train.columns if col.startswith('job_desc_')]

train['has_description'] = (train[job_desc_columns].sum(axis=1) != 0).astype(int)
test['has_description'] = (test[job_desc_columns].sum(axis=1) != 0).astype(int)

#### PCA dla kolumn job_desc

In [ ]:
pca_n_components = 6
pca_columns = [f'pca_job_desc_{i}' for i in range(pca_n_components)]

standard_scaler = StandardScaler()
train[job_desc_columns] = standard_scaler.fit_transform(train[job_desc_columns])
test[job_desc_columns] = standard_scaler.transform(test[job_desc_columns])

job_desc_pca = PCA(n_components=pca_n_components, random_state=42).fit(train[job_desc_columns])

train_pca_decomposed = job_desc_pca.transform(train[job_desc_columns])
test_pca_decomposed = job_desc_pca.transform(test[job_desc_columns])

train.drop(columns=job_desc_columns, inplace=True)
test.drop(columns=job_desc_columns, inplace=True)

train = pd.concat([train, pd.DataFrame(data=train_pca_decomposed, columns=pca_columns)], axis=1)
test = pd.concat([test, pd.DataFrame(data=test_pca_decomposed, columns=pca_columns)], axis=1)

#### Ilość zachowanych informacji przez każdą składową główną PCA

In [ ]:
preserved_info = pd.DataFrame(data={
    'components': pca_columns,
    'variance': job_desc_pca.explained_variance_ratio_ * 100
})

plt.figure(figsize=(12, 7))
sns.barplot(
    data=preserved_info,
    x='components',
    y='variance',
    color='skyblue',
    edgecolor='black'
)

plt.xlabel('Składowe główne', fontsize=14)
plt.ylabel('Procent zachowanych informacji [%]', fontsize=14)
plt.title('Analiza PCA: Procent wariancji wyjaśnionej przez składowe główne', fontsize=16, pad=20)

for i, v in enumerate(preserved_info['variance']):
    plt.text(i, v + 0.5, f'{v:.1f}%', ha='center', fontsize=12)

plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()

### <center>Miesiąc i rok</center>

In [ ]:
(train['job_posted_date'].isna().sum(), test['job_posted_date'].isna().sum())

In [ ]:
train['job_posted_date'] = train['job_posted_date'].fillna(train['job_posted_date'].mode()[0])

train['job_posted_date'] = pd.to_datetime(train['job_posted_date'], format='%Y/%m')
train['month'] = train['job_posted_date'].dt.month
train['year'] = train['job_posted_date'].dt.year
train.drop(columns=['job_posted_date'], inplace=True)

test['job_posted_date'] = pd.to_datetime(test['job_posted_date'], format='%Y/%m')
test['month'] = test['job_posted_date'].dt.month
test['year'] = test['job_posted_date'].dt.year
test.drop(columns=['job_posted_date'], inplace=True)

### <center>Porównanie wybranych kolumn kategorycznych w train/test</center>

#### <center>Stan w którym jest oferta pracy "job_state"</center>

In [ ]:
c_job_state

#### <center>WYKRES: Region USA, a wysokość pensji</center>

In [ ]:
def plot_bar(column, width, xlabel, ylabel, title, main_ax=None, fontsize=16, show_legend=True, rotation=90):
    salary_types = train['salary_category'].unique()
    show_every_column = train.groupby([column, 'salary_category']).size().unstack(fill_value=0).reindex(columns=salary_types, fill_value=0).stack().reset_index(name='Count')

    x_axis = np.arange(len(show_every_column[column].unique()))
    my_xticks = [x_axis - width, x_axis, x_axis + width]
    colors = ['gold', 'silver', 'brown']

    if main_ax is None:
        f, a = plt.subplots(figsize=(10, 8))
    else:
        a = main_ax

    for i, salary in enumerate(salary_types):
        a.bar(
            x=my_xticks[i],
            height=show_every_column.loc[(show_every_column['salary_category'] == salary), 'Count'],
            width=width,
            label=salary,
            color=colors[i],
            edgecolor='black',
            alpha=1
        )

    a.grid(True)
    a.set_xticks(x_axis)
    a.set_xticklabels(show_every_column[column].unique(), rotation=rotation)
    a.set_xlabel(xlabel, fontsize=fontsize)
    a.set_ylabel(ylabel, fontsize=fontsize)
    a.set_title(title, fontsize=fontsize)
    if show_legend:
        a.legend(title='Kategoria wynagrodzenia')


plot_bar(
    column='job_state',
    width=0.2,
    xlabel='Stan',
    ylabel='Liczba ofert',
    title='Kategoria zarobkowa w stanach'
)

#### Zmiana stanów na regiony USA

In [ ]:
west_states = [
    'AK', 'AZ', 'CA', 'CO', 'HI', 'ID',
    'MT', 'NV', 'NM', 'OR', 'UT', 'WA', 'WY'
]

central_states = [
    'AR', 'IA', 'IL', 'IN', 'KS', 'KY', 'LA',
    'MI', 'MN', 'MO', 'ND', 'NE', 'OH',
    'OK', 'SD', 'TX', 'WI'
]

east_states = [
    'AL', 'CT', 'DE', 'FL', 'GA', 'MA', 'MD',
    'ME', 'MS', 'NC', 'NH', 'NJ', 'NY', 'PA',
    'RI', 'SC', 'TN', 'VA', 'VT', 'WV', 'DC'
]

def map_state_to_region(state):
    if state in west_states:
        return 'West'
    elif state in central_states:
        return 'Central'
    elif state in east_states:
        return 'East'
    else:
        return 'None'

train['job_region'] = train['job_state'].apply(map_state_to_region)
test['job_region'] = test['job_state'].apply(map_state_to_region)

#### <center>WYKRES: Region USA, a wysokość pensji</center>

In [ ]:
plot_bar(
    column='job_region',
    width=0.2,
    xlabel='Region',
    ylabel='Liczba ofert',
    title='Kategoria zarobkowa w regionach'
)

#### Co zrobić z job_state?

Testowano:
- usunięcie -> tak sobie
- zmiana na "job_region" i kodowanie one-hot -> narazie najlepiej

In [ ]:
job_region_one_hot = OneHotEncoder(sparse_output=False, dtype=int)
job_region_one_hot.fit(train[['job_region']])

train = pd.concat([train, pd.DataFrame(job_region_one_hot.transform(train[['job_region']]), columns=job_region_one_hot.categories_[0])], axis=1)
test = pd.concat([test, pd.DataFrame(job_region_one_hot.transform(test[['job_region']]), columns=job_region_one_hot.categories_[0])], axis=1)

train.drop(columns=['job_state', 'job_region'], inplace=True)
test.drop(columns=['job_state', 'job_region'], inplace=True)

#### <center>Stanowisko pracy "job_title"</center>

In [ ]:
c_job_title

In [ ]:
(train['job_title'].isna().sum(), test['job_title'].isna().sum())

#### <center>WYKRES: Stanowisko pracy a wysokość pensji</center>

In [ ]:
plot_bar(
    column='job_title',
    width=0.2,
    xlabel='Stanowisko',
    ylabel='Liczba ofert',
    title='Kategorie zarobkowe dla każdej oferty pracy'
)

#### Co zrobić z job_title?

Testowane:
- kodowanie one-hot -> nie poprawia
- usunięcie -> narazie najlepiej
- pogrupować tak żeby była przewaga jakiś klas z 'salary_category'. Stworzenie 'job_title_earning' i kodowanie one-hot -> nie poprawia
- zmiana na binary 'is_others_job_title' -> działa, ale zbadać wpływ na ostateczny wynik

In [ ]:
train.loc[train['job_title'] == 'Others', 'is_others_job_title'] = 1
test.loc[train['job_title'] == 'Others', 'is_others_job_title'] = 1

train.loc[train['job_title'] != 'Others', 'is_others_job_title'] = 0
test.loc[train['job_title'] != 'Others', 'is_others_job_title'] = 0

train['is_others_job_title'] = train['is_others_job_title'].astype(int)

train.drop(columns=['job_title'], inplace=True)
test.drop(columns=['job_title'], inplace=True)

#### <center>WYKRES: Rodzaj stanowiska pracy a wysokość pensji</center>

In [ ]:
plot_bar(
    column='is_others_job_title',
    width=0.2,
    xlabel='is_others_job_title',
    ylabel='Liczba ofert',
    title='Kategorie zarobkowe dla typu oferty',
    rotation=0
)

#### <center>Kolumny "feature_"</center>

In [ ]:
feature_columns = [col for col in train.columns if col.startswith('feature_')]

train[feature_columns].info()

In [ ]:
train.info()

#### <center>WYKRES: Podział wartości kolumn feature według wysokość pensji</center>

In [ ]:
exclude_float = train[feature_columns].select_dtypes(exclude=np.float64).columns
fig, ax = plt.subplots(nrows=5, ncols=2, figsize=(12, 20))

for i, col in enumerate(exclude_float):
    x, y = divmod(i, 2)
    plot_bar(
        column=col,
        width=0.2,
        xlabel=col,
        ylabel='Liczba ofert',
        title='',
        main_ax=ax[x, y],
        fontsize=12,
        show_legend=True,
        rotation=0
    )

#### feature_1

In [ ]:
grouped_feature_1 = train.groupby(['salary_category', 'feature_1']).size().unstack(fill_value=0)

feature_1_encoder = LabelEncoder().fit(train['feature_1'])
train['feature_1'] = feature_1_encoder.transform(train['feature_1'])
test['feature_1'] = feature_1_encoder.transform(test['feature_1'])

grouped_feature_1

#### Korelacja kolumn feature

In [ ]:
plt.figure(figsize=(10, 6))
sns.heatmap(
    train.loc[train['feature_10'].notna(), feature_columns].corr(),
    annot=True,
    fmt='.2f',
    cmap='viridis',
    linewidths=0.5,
    linecolor='black',
    square=True,
    cbar=True
)

#### feature_2 (bo jest najwyżej w importance dla xgb, lgb i rf)

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(
    data=train,
    x='feature_2',
    hue='salary_category',
    kde=True,
    palette='Set2',
    multiple='stack',
    bins=100,
)

plt.title('Rozkład feature_2 według kategorii wynagrodzenia', fontsize=14)
plt.xlabel('feature_2', fontsize=12)
plt.ylabel('Liczba przypadków', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

#### feature_10 (bo jest najwięcej NaN)

W feature_10 okres 06-08/2024 ma najwięcej brakujących wartości dla zbioru treningowego.

Tak samo jest w zbiorze testowym.

In [ ]:
train_grouped_feature_10 = train.loc[train['feature_10'].isna(), ['month', 'year']]

train_grouped_feature_10.groupby(['year', 'month']).size().sort_values(ascending=False).head(n=3)

In [ ]:
test_grouped_feature_10 = test.loc[test['feature_10'].isna(), ['month', 'year']]

test_grouped_feature_10.groupby(['year', 'month']).size().sort_values(ascending=False).head(n=3)

#### Kodowanie LabelEncoder na salary_category

In [ ]:
salary_category = train['salary_category']
salary_category_encoder = LabelEncoder().fit(salary_category)
train['salary_category'] = salary_category_encoder.transform(train['salary_category'])

#### Korelacja dla całego zbioru

In [ ]:
plt.figure(figsize=(10, 6))
sns.heatmap(
    train.corr(),
    annot=True,
    fmt='.2f',
    cmap='viridis',
    linewidths=0.5,
    linecolor='black',
    square=True,
    cbar=True
)

#### Wartości korelacji dla kolumny salary_category

Usunięcie kolumn mało informatywnych.

Usuń kolumny wysoko skorelowane.

In [ ]:
test_obs = test['obs']
train.drop(columns=['obs'], inplace=True)
test.drop(columns=['obs'], inplace=True)

corr = train.corr()[['salary_category']]

columns_to_drop = corr.loc[(corr['salary_category'] < 0.02) & (corr['salary_category'] > -0.02), :].index.values

train['salary_category'] = salary_category_encoder.inverse_transform(train['salary_category'])
train.drop(columns=columns_to_drop, inplace=True)
test.drop(columns=columns_to_drop, inplace=True)

test = pd.concat([test_obs, test], axis=1)

columns_to_drop

In [ ]:
def drop_high_correlated_columns():
    matrix = train.corr(numeric_only=True).abs()
    upper_t = matrix.where(np.triu(np.ones_like(matrix, dtype=np.bool_), k=1))
    return [col for col in upper_t.columns if any(upper_t[col] > 0.9)]


high_correlated_columns = drop_high_correlated_columns()
train.drop(columns=high_correlated_columns, inplace=True)
test.drop(columns=high_correlated_columns, inplace=True)

high_correlated_columns

In [ ]:
train.head()

In [ ]:
test.head()

## Przybliżenie train do test

- ucięcie przedziałów na train aby zgadzały się z test na podstawie wyniku testu U Manna Whitneya -> nie poprawia
- usunięcie tych kolumn -> nie testowano

In [ ]:
statistical_tests = pd.DataFrame(
    data={
        'columns': train.columns[1:],
        'umann_pvalue': [mannwhitneyu(train[column], test[column], nan_policy='omit', use_continuity=False, method='asymptotic', alternative='two-sided').pvalue for column in train.columns if column != 'salary_category']
    }
)

check_for_diff = statistical_tests.loc[statistical_tests['umann_pvalue'] <= 0.05, :].sort_values(by='umann_pvalue')

In [ ]:
check_for_diff

In [ ]:
# def cut_ranges(column):
#     test_min, test_max = test[column].min(), test[column].max()
#     return train[(train[column] >= test_min) & (train[column] <= test_max)].reset_index(drop=True)
#
# for column in check_for_diff['columns']:
#     train = cut_ranges(column)
#
# train.info()

train.drop(columns=check_for_diff['columns'], inplace=True)
test.drop(columns=check_for_diff['columns'], inplace=True)

#### Zapisz przygotowane dane

In [ ]:
if 'prepared' not in os.listdir(os.getcwd()):
    os.mkdir('prepared')

train.to_csv(os.path.join('prepared', 'train.csv'), index=False)
test.to_csv(os.path.join('prepared', 'test.csv'), index=False)